In [101]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("solutions").getOrCreate()

In [102]:
from pyspark.sql.functions import sum , col 


In [103]:
#load datasets 

data_path = "./data/revised_final_data.csv"  
df = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/goods_classification.csv"  
df_goods = spark.read.csv(data_path, header=True, inferSchema=True)

data_path = "./data/country_classification.csv"  
df_country = spark.read.csv(data_path, header=True, inferSchema=True)
data_path = "./data/services_classification.csv"  
df_services = spark.read.csv(data_path, header=True, inferSchema=True)



In [104]:
df.show()

+----------+-------+----+------------+------------+-------------+------+
|  time_ref|account|code|country_code|product_type|        value|status|
+----------+-------+----+------------+------------+-------------+------+
|2023-06-30|Exports|  00|          AE|       Goods| 2.82028909E8|     F|
|2023-06-30|Exports|  00|          AG|       Goods|     351919.0|     F|
|2023-06-30|Exports|  00|          AI|       Goods|      84762.0|     F|
|2023-06-30|Exports|  00|          AL|       Goods|       3463.0|     F|
|2023-06-30|Exports|  00|          AM|       Goods|     679586.0|     F|
|2023-06-30|Exports|  00|          AO|       Goods|     464583.0|     F|
|2023-06-30|Exports|  00|          AR|       Goods|    5943055.0|     F|
|2023-06-30|Exports|  00|          AS|       Goods|  1.1622992E7|     F|
|2023-06-30|Exports|  00|          AT|       Goods|  1.1202334E7|     F|
|2023-06-30|Exports|  00|          AU|       Goods|2.272665701E9|     F|
|2023-06-30|Exports|  00|          AW|       Goods|

In [105]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- status: string (nullable = true)



In [106]:
df = df.withColumn('value', col('value').cast('int'))

In [107]:
df.printSchema()

root
 |-- time_ref: date (nullable = true)
 |-- account: string (nullable = true)
 |-- code: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- status: string (nullable = true)



<h2>Question1 : Calculate the total export value for each country in the 'revised_final' table. Show the top 10 countries with the highest total export values.
 </h2>

In [108]:

result = df.where(col('account')=='Exports').groupBy('country_code').agg(sum('value').alias('total_export_value'))
result = result.join(df_country,result['country_code']==df_country['country_code'],'inner')
result = result.select('country_label',df['country_code'],'total_export_value')
result = result.orderBy(result.total_export_value.desc()).limit(10)
result.show()

+--------------------+------------+------------------+
|       country_label|country_code|total_export_value|
+--------------------+------------+------------------+
|China, People's R...|          CN|      168541418942|
|           Australia|          AU|      124044791511|
|United States of ...|          US|      106261562532|
|               Japan|          JP|       40647876106|
|  Korea, Republic of|          KR|       24767856475|
|      United Kingdom|          GB|       22290125277|
|           Singapore|          SG|       18358723868|
|           Indonesia|          ID|       15861041776|
|              Taiwan|          TW|       15548494077|
|Hong Kong (Specia...|          HK|       12708136935|
+--------------------+------------+------------------+



+----------+-------+----+------------+------------+----------+------+
|  time_ref|account|code|country_code|product_type|     value|status|
+----------+-------+----+------------+------------+----------+------+
|2023-06-30|Exports|  00|          AE|       Goods| 282028909|     F|
|2023-06-30|Exports|  00|          AG|       Goods|    351919|     F|
|2023-06-30|Exports|  00|          AI|       Goods|     84762|     F|
|2023-06-30|Exports|  00|          AL|       Goods|      3463|     F|
|2023-06-30|Exports|  00|          AM|       Goods|    679586|     F|
|2023-06-30|Exports|  00|          AO|       Goods|    464583|     F|
|2023-06-30|Exports|  00|          AR|       Goods|   5943055|     F|
|2023-06-30|Exports|  00|          AS|       Goods|  11622992|     F|
|2023-06-30|Exports|  00|          AT|       Goods|  11202334|     F|
|2023-06-30|Exports|  00|          AU|       Goods|2147483647|     F|
|2023-06-30|Exports|  00|          AW|       Goods|    362396|     F|
|2023-06-30|Exports|